## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [15]:
from agents import (
    Agent,
    Model,
    ModelProvider,
    OpenAIChatCompletionsModel,
    RunConfig,
    Runner,
    function_tool,
    set_tracing_disabled,
    set_tracing_export_api_key,
    trace
)
from openai import OpenAI, AsyncOpenAI
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from tavily import TavilyClient
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown




In [16]:
load_dotenv(override=True)

True

In [17]:
# ==========================
# Environment Variables
# ==========================

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("GOOGLE_API_KEY") or ""   # <-- Your Gemini API key
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
MODEL_NAME = "gemini-2.0-flash"               # <-- Gemini model


# ==========================
# Create a Gemini Client
# ==========================
# This tells the SDK to use Gemini instead of OpenAI
client = AsyncOpenAI(base_url=BASE_URL, api_key=API_KEY)

# Optional: disable OpenAI tracing if you don’t have an OpenAI key
# set_tracing_disabled(disabled=True)

tracing_api_key = os.environ.get("OPENAI_API_KEY")
set_tracing_export_api_key(tracing_api_key)

# ==========================
# Custom Model Provider
# ==========================
class GeminiModelProvider(ModelProvider):
    def get_model(self, model_name: str | None) -> Model:
        return OpenAIChatCompletionsModel(
            model=model_name or MODEL_NAME,
            openai_client=client   # <-- use our Gemini client here
        )

GEMINI_PROVIDER = GeminiModelProvider()


In [4]:
# ==========================
# Tools
# ==========================

tavily = TavilyClient(api_key=TAVILY_API_KEY)

# Wrap Tavily search in a tool
@function_tool
def tavily_search(query: str) -> str:
    """
    Perform a web search using Tavily and return the best results.
    """
    result = tavily.search(query=query, max_results=10)
    return result


In [5]:
gemini_model_name = "gemini-2.0-flash"

INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[tavily_search],
    model_settings=ModelSettings(tool_choice="required"),
)

In [17]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(
        search_agent,
        message,
        run_config=RunConfig(model_provider=GEMINI_PROVIDER),
    )

display(Markdown(result.final_output))

AI agent frameworks are expanding, offering new tools for developers. Key frameworks mentioned for 2025 include LangChain, LangGraph, CrewAI, Microsoft Semantic Kernel, and AutoGen. These frameworks serve as building blocks for developing, deploying, and managing AI agents, equipped with built-in features.


### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [6]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    output_type=WebSearchPlan,
)

In [21]:
message = "Latest Causal AI develpments in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message, run_config=RunConfig(model_provider=GEMINI_PROVIDER))
    print(result.final_output)

searches=[WebSearchItem(reason='To find publications specifically discussing advancements in causal AI anticipated or predicted for 2025.', query='"causal AI" "future trends" 2025'), WebSearchItem(reason='To identify research papers, articles, or industry reports forecasting developments in causal inference and machine learning expected around 2025.', query='"causal inference" "machine learning" "future developments" 2025'), WebSearchItem(reason='To uncover predictions and expectations from experts in the field regarding the evolution of causal AI technologies and applications leading up to 2025.', query='"causal AI" "expert predictions" 2025')]


In [ ]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(os.getenv('sender_email')) # Change this to your verified email
    to_email = To(os.getenv('receiver_email')) # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [8]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11c10c900>, strict_json_schema=True, is_enabled=True)

In [9]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email]
)



In [10]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    output_type=ReportData
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [11]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}", run_config=RunConfig(model_provider=GEMINI_PROVIDER))
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input, run_config=RunConfig(model_provider=GEMINI_PROVIDER))
    return result.final_output

### The next 2 functions write a report and email it

In [12]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input, run_config=RunConfig(model_provider=GEMINI_PROVIDER))
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report, run_config=RunConfig(model_provider=GEMINI_PROVIDER))
    print("Email sent")
    return report

### Showtime!

In [18]:
query ="promising research ideas for Quantitative Finance, Machine Learning and NLP for 2025 onwards"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")




Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>